In [4]:
# !pip install wilds

In [5]:
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import torchvision.models as models
from wilds.common.data_loaders import get_eval_loader
import torch
from torch import nn
from tqdm import tqdm

In [6]:
# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="camelyon17", download=True)


You can also download the dataset manually at https://wilds.stanford.edu/downloads.


  0%|          | 0/10658709504 [00:00<?, ?Byte/s]

Extracting data/camelyon17_v1.0/archive.tar.gz to data/camelyon17_v1.0

It took 43.46 minutes to download and uncompress the dataset.



In [7]:
BATCH_SIZE = 32
FRACTION = 0.2

# Get the training set
train_data = dataset.get_subset(
    "train",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(train_data)) #302436 initially
# Prepare the standard data loader
train_loader = get_train_loader("standard", train_data, batch_size=BATCH_SIZE)

"""
# (Optional) Load unlabeled data
dataset = get_dataset(dataset="camelyon17", download=True, unlabeled=True)
unlabeled_data = dataset.get_subset(
    "test_unlabeled",
    transform=transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    ),
)
unlabeled_loader = get_train_loader("standard", unlabeled_data, batch_size=16)
"""
"""
# Train loop
for labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):
    x, y, metadata = labeled_batch
    unlabeled_x, unlabeled_metadata = unlabeled_batch
    ...
"""

60487


'\n# Train loop\nfor labeled_batch, unlabeled_batch in zip(train_loader, unlabeled_loader):\n    x, y, metadata = labeled_batch\n    unlabeled_x, unlabeled_metadata = unlabeled_batch\n    ...\n'

In [8]:
# Get the test set
id_val_data = dataset.get_subset(
    "id_val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(id_val_data))

# Prepare the evaluation data loader
id_val_loader = get_eval_loader("standard", id_val_data, batch_size=BATCH_SIZE)


6712


In [9]:
# Get the test set
val_data = dataset.get_subset(
    "val",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(val_data))

# Prepare the evaluation data loader
val_loader = get_eval_loader("standard", val_data, batch_size=BATCH_SIZE)

6981


In [27]:
# Get the test set
test_data = dataset.get_subset(
    "test",
    frac = FRACTION,
    transform=transforms.Compose(
        [
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])]
    ),
)

print(len(test_data))

# Prepare the evaluation data loader
test_loader = get_eval_loader("standard", test_data, batch_size=BATCH_SIZE)

17011


In [11]:

# load the ResNet-18 model, with weights pretrained on ImageNet
resnet18_pretrained = models.resnet18(pretrained=True)

num_params = 0
print("Model's parameters: ")
for n, p in resnet18_pretrained.named_parameters():
    print('\t', n, ': ', p.size())
    num_params += p.numel()
print("Number of model parameters: ", num_params)

"""
formula [(W−K+2P)/S]+1.

W is the input volume
K is the Kernel size
P is the padding
S is the stride

"""

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

'\nformula [(W−K+2P)/S]+1.\n\nW is the input volume\nK is the Kernel size\nP is the padding\nS is the stride\n\n'

In [12]:
# function counting the number of parameters and the number of trainable parameters of a model
# optionally, it will also display the layers
def check_model_parameters(model, display_layers=False):
  num_params = 0
  num_trainable_params = 0
  if display_layers==True:
    print("Model's parameters: ")
  for n, p in model.named_parameters():
      if display_layers == True:
        print('\t', n, ': ', p.size())
      num_params += p.numel()
      if p.requires_grad:
        num_trainable_params += p.numel()
  print("Number of model parameters: ", num_params)
  print("Number of trainable parameters: ", num_trainable_params)

In [13]:
# freeze the model parameters

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

# freeze all the layers
for param in resnet18_pretrained.parameters():
  param.requires_grad = False

# check the number of parameters and the number of trainable parameters
check_model_parameters(resnet18_pretrained, display_layers=False)

Number of model parameters:  11689512
Number of trainable parameters:  11689512
Number of model parameters:  11689512
Number of trainable parameters:  0


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_epochs = 5

In [15]:

resnet18_pretrained.fc = nn.Linear(in_features=512, out_features=2, bias=True)

check_model_parameters(resnet18_pretrained, display_layers=True)

Model's parameters: 
	 conv1.weight :  torch.Size([64, 3, 7, 7])
	 bn1.weight :  torch.Size([64])
	 bn1.bias :  torch.Size([64])
	 layer1.0.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn1.weight :  torch.Size([64])
	 layer1.0.bn1.bias :  torch.Size([64])
	 layer1.0.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.0.bn2.weight :  torch.Size([64])
	 layer1.0.bn2.bias :  torch.Size([64])
	 layer1.1.conv1.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn1.weight :  torch.Size([64])
	 layer1.1.bn1.bias :  torch.Size([64])
	 layer1.1.conv2.weight :  torch.Size([64, 64, 3, 3])
	 layer1.1.bn2.weight :  torch.Size([64])
	 layer1.1.bn2.bias :  torch.Size([64])
	 layer2.0.conv1.weight :  torch.Size([128, 64, 3, 3])
	 layer2.0.bn1.weight :  torch.Size([128])
	 layer2.0.bn1.bias :  torch.Size([128])
	 layer2.0.conv2.weight :  torch.Size([128, 128, 3, 3])
	 layer2.0.bn2.weight :  torch.Size([128])
	 layer2.0.bn2.bias :  torch.Size([128])
	 layer2.0.downsample.0.weight :  torch.Size([12

In [16]:
def train_epoch(model, train_dataloader, loss_crt, optimizer, device):
    """
    model: Model object
    train_dataloader: DataLoader over the training dataset
    loss_crt: loss function object
    optimizer: Optimizer object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch training loss, which is an average over the individual batch
       losses
    """
    model.train()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(train_dataloader)
    for batch_idx, batch in tqdm(enumerate(train_dataloader)):
        # shape: batch_size x 1 x 28 x 28, batch_size x 1
        # print("Train")
        # print(batch)
        batch_img, batch_labels, _ = batch
        # move data to GPU
        batch_img = batch_img.to(device)
        batch_labels = batch_labels.to(device)

        # initialize as zeros all the gradients of the model
        model.zero_grad()

        # get predictions from the FORWARD pass
        # shape: batch_size x 10
        output = model(batch_img)

        loss = loss_crt(output, batch_labels.squeeze())
        loss_scalar = loss.item()

        # BACKPROPAGATE the gradients
        loss.backward()
        # use the gradients to OPTIMISE the model
        optimizer.step()

        epoch_loss += loss_scalar

        pred = output.argmax(dim=1, keepdim=True)
        epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

def eval_epoch(model, val_dataloader, loss_crt, device):
    """
    model: Model object
    val_dataloader: DataLoader over the validation dataset
    loss_crt: loss function object
    device: torch.device('cpu) or torch.device('cuda')

    The function returns:
     - the epoch validation loss, which is an average over the individual batch
       losses
    """
    model.eval()
    epoch_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(val_dataloader)
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(val_dataloader)):
            # print("Eval")
            # print(batch)
            # shape: batch_size x 3 x 28 x 28, batch_size x 1
            batch_img, batch_labels, _ = batch
            current_batch_size = batch_img.size(0)

            # move data to GPU
            batch_img = batch_img.to(device)
            batch_labels = batch_labels.to(device)

            # batch_size x 10
            output = model(batch_img)

            loss = loss_crt(output, batch_labels.squeeze())
            loss_scalar = loss.item()

            epoch_loss += loss_scalar

            pred = output.argmax(dim=1, keepdim=True)
            epoch_accuracy += pred.eq(batch_labels.view_as(pred)).float().mean().item()

    epoch_loss = epoch_loss/num_batches
    epoch_accuracy = 100. * epoch_accuracy/num_batches
    return epoch_loss, epoch_accuracy

In [17]:
resnet18_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnet18_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnet18_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

1891it [02:49, 11.18it/s]
210it [00:16, 12.36it/s]



Epoch 1
train loss: 0.53807365, accuracy: 86.91333950
id_val loss: 0.40365150, accuracy: 91.33928571


1891it [01:48, 17.38it/s]
210it [00:10, 20.39it/s]



Epoch 2
train loss: 0.57412050, accuracy: 87.37511332
id_val loss: 0.54578753, accuracy: 89.42956351


1891it [01:35, 19.85it/s]
210it [00:10, 20.26it/s]



Epoch 3
train loss: 0.55083623, accuracy: 87.34300635
id_val loss: 0.47509170, accuracy: 90.91765875


1891it [01:36, 19.69it/s]
210it [00:11, 18.65it/s]



Epoch 4
train loss: 0.52754328, accuracy: 87.52809360
id_val loss: 0.84469664, accuracy: 81.07142857


1891it [01:35, 19.71it/s]
210it [00:10, 20.48it/s]


Epoch 5
train loss: 0.53344837, accuracy: 87.37251643
id_val loss: 0.57009999, accuracy: 89.49404762


In [33]:
def run_eval_ood(model, loader, loss_criterion, device, eval_type):
  losses = []
  accuracies = []

  loss, accuracy = eval_epoch(model, loader, loss_criterion, device)
  losses.append(loss)
  accuracies.append(accuracy)
  print(eval_type + ' loss: %10.8f, accuracy: %10.8f'%(loss, accuracy))

In [34]:
run_eval_ood(resnet18_pretrained, val_loader, loss_criterion, device, "val")

219it [00:12, 17.01it/s]

val loss: 1.25034301, accuracy: 80.29394977


In [35]:
run_eval_ood(resnet18_pretrained, test_loader, loss_criterion, device, "test")

532it [00:28, 18.81it/s]

test loss: 2.02949556, accuracy: 69.68984962


In [19]:
densenet121_pretrained = models.densenet121(pretrained=True)

for param in densenet121_pretrained.parameters():
  param.requires_grad = False

check_model_parameters(densenet121_pretrained, display_layers=False)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

Number of model parameters:  7978856
Number of trainable parameters:  0


In [20]:

densenet121_pretrained.classifier = nn.Linear(in_features=1024, out_features=2, bias=True)
check_model_parameters(densenet121_pretrained, display_layers=False)

Number of model parameters:  6955906
Number of trainable parameters:  2050


In [21]:
densenet121_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(densenet121_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()

# evaluate the initial model
# val_loss, al_accuracy = eval_epoch(resnet18_pretrained, id_val_loader, loss_criterion, device)
# print('Validation performance before finetuning -- loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))

# finetune the model
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
id_val_losses = []
id_val_accuracies = []

for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(densenet121_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(densenet121_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


1891it [02:20, 13.41it/s]
210it [00:13, 15.01it/s]



Epoch 1
train loss: 0.48657711, accuracy: 88.96818577
id_val loss: 0.29367816, accuracy: 92.55456351


1891it [02:20, 13.42it/s]
210it [00:14, 14.93it/s]



Epoch 2
train loss: 0.45201103, accuracy: 89.74087784
id_val loss: 0.41586063, accuracy: 90.24305557


1891it [02:20, 13.51it/s]
210it [00:14, 14.99it/s]



Epoch 3
train loss: 0.42659008, accuracy: 90.30345811
id_val loss: 0.33243015, accuracy: 93.09523810


1891it [02:20, 13.47it/s]
210it [00:13, 15.14it/s]



Epoch 4
train loss: 0.46719336, accuracy: 89.95240613
id_val loss: 0.36094481, accuracy: 91.12599208


1891it [02:19, 13.51it/s]
210it [00:13, 15.17it/s]


Epoch 5
train loss: 0.46445901, accuracy: 90.06808567
id_val loss: 0.30863111, accuracy: 92.72321429


In [36]:
run_eval_ood(densenet121_pretrained, val_loader, loss_criterion, device, "val")

219it [00:14, 15.05it/s]

val loss: 0.76147554, accuracy: 84.17522831


In [37]:
run_eval_ood(densenet121_pretrained, test_loader, loss_criterion, device, "test")

532it [00:36, 14.74it/s]

test loss: 1.23863125, accuracy: 75.68139098


In [38]:
torch.save(resnet18_pretrained.state_dict(), "resnet18_pretrained.pt")

In [39]:
torch.save(densenet121_pretrained.state_dict(), "densenet121_pretrained.pt")

In [42]:
resnetxt50_pretrained = models.resnext50_32x4d(pretrained=True)

# freeze all the layers
for param in resnetxt50_pretrained.parameters():
  param.requires_grad = False

resnetxt50_pretrained.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

resnetxt50_pretrained.to(device)

# create a SGD optimizer
optimizer = torch.optim.SGD(resnetxt50_pretrained.parameters(), lr=0.01, momentum=0.9)

# set up loss function
loss_criterion = nn.CrossEntropyLoss()


# finetune the model
train_losses = []
train_accuracies = []
id_val_losses = []
id_val_accuracies = []
for epoch in range(1, num_epochs+1):
  train_loss, train_accuracy = train_epoch(resnetxt50_pretrained, train_loader, loss_criterion, optimizer, device)
  val_loss, val_accuracy = eval_epoch(resnetxt50_pretrained, id_val_loader, loss_criterion, device)
  train_losses.append(train_loss)
  id_val_losses.append(val_loss)
  train_accuracies.append(train_accuracy)
  id_val_accuracies.append(val_accuracy)
  print('\nEpoch %d'%(epoch))
  print('train loss: %10.8f, accuracy: %10.8f'%(train_loss, train_accuracy))
  print('id_val loss: %10.8f, accuracy: %10.8f'%(val_loss, val_accuracy))


1891it [02:17, 13.75it/s]
210it [00:13, 15.86it/s]



Epoch 1
train loss: 0.47027896, accuracy: 87.19569200
id_val loss: 0.30470829, accuracy: 91.84523810


1891it [02:06, 14.97it/s]
210it [00:13, 15.64it/s]



Epoch 2
train loss: 0.46927865, accuracy: 87.99978281
id_val loss: 0.34837296, accuracy: 91.84027780


1891it [02:08, 14.75it/s]
210it [00:13, 16.06it/s]



Epoch 3
train loss: 0.48041085, accuracy: 88.37090164
id_val loss: 0.31739704, accuracy: 92.04861113


1891it [02:05, 15.05it/s]
210it [00:13, 15.85it/s]



Epoch 4
train loss: 0.43898939, accuracy: 89.03995430
id_val loss: 0.68546015, accuracy: 84.14682542


1891it [02:05, 15.11it/s]
210it [00:13, 15.74it/s]


Epoch 5
train loss: 0.41872957, accuracy: 89.25998149
id_val loss: 0.37399658, accuracy: 91.43353176


In [43]:
run_eval_ood(resnetxt50_pretrained, val_loader, loss_criterion, device, "val")

219it [00:13, 15.79it/s]

val loss: 0.96758571, accuracy: 79.36643836


In [44]:
run_eval_ood(resnetxt50_pretrained, test_loader, loss_criterion, device, "test")

532it [00:34, 15.59it/s]

test loss: 1.92752245, accuracy: 63.33411654


In [45]:
torch.save(densenet121_pretrained.state_dict(), "resnetxt50_pretrained.pt")